# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time
import os
import argparse
from torchvision import datasets, models, transforms, utils
from torch.autograd import Variable
import torch.nn.functional as F
import copy
from PIL import Image
import matplotlib.pyplot as plt
import json
from collections import OrderedDict

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'
cuda=torch.cuda.is_available()

In [3]:
# TODO: Define your transforms for the training, validation, and testing sets
data_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder

image_datasets = datasets.ImageFolder(data_dir + '/train', transform=data_transforms)
valid_datasets = datasets.ImageFolder(data_dir + '/valid', transform=valid_transforms)
test_datasets = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders

dataloaders = torch.utils.data.DataLoader(image_datasets, batch_size=64, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_datasets, batch_size=32)
testloader = torch.utils.data.DataLoader(test_datasets, batch_size=32)


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [5]:
# TODO: Build and train your network  
model=models.vgg16(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
    
initial_input= model.classifier[0].in_features
classifier= nn.Sequential(OrderedDict([
                              ('fc1', nn.Linear(initial_input,512)),
                              ('relu', nn.ReLU()),
                              ('drpot', nn.Dropout(p=0.5)),
                              ('hidden', nn.Linear(512, 100)), 
                              ('fc2', nn.Linear(100,102)),
                              ('output', nn.LogSoftmax(dim=1)),
                              ]))
model.classifier = classifier

epochs=10
if torch.cuda.is_available():
    model.cuda()
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.classifier.parameters(),lr=0.001)

def train(model, dataloader, validloader, criterion, optimizer, epochs, print_every,device='cpu'): 
    i = 0
    if device and torch.cuda.is_available():
        model.cuda()
    else:
        print('Go without cuda')
    for e in range(epochs):
        model.train()
        for images, labels in iter(dataloader):
            i += 1
            
            if device and torch.cuda.is_available():
                images = images.cuda()
                labels = labels.cuda()
            else:
                images =Variable(images)
                labels =Variable(labels)
            optimizer.zero_grad()
            result = model.forward(images)
            loss_value = criterion(result, labels)
            loss_value.backward()
            optimizer.step()
            if i % print_every == 0:
                model.eval()
                accuracy=0
                valid_loss=0
                for images, labels in iter(validloader):
            
                    if device and torch.cuda.is_available():
                        images =images.cuda()
                        labels = labels.cuda()
                    else:
                        images = Variable(images)
                        labels =  Variable(labels)
                    with torch.no_grad():
                        result = model.forward(images)
                        valid_loss=criterion(result, labels).data.item()
                        prob = torch.exp(result).data
                        equality = (labels.data == prob.max(1)[1])
                        accuracy += equality.type_as(torch.FloatTensor()).mean()

                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Validation Loss: {:.3f}.. ".format(valid_loss/len(validloader)),
                      "Accuracy: {:.3f}".format(accuracy/len(validloader)))
                running_loss = 0
                model.train()
model=train(model, dataloaders, validloader, criterion, optimizer, 5, 20,'gpu')
print(model)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:09<00:00, 57039291.93it/s]


Epoch: 1/5..  Validation Loss: 0.171..  Accuracy: 0.269
Epoch: 1/5..  Validation Loss: 0.113..  Accuracy: 0.501
Epoch: 1/5..  Validation Loss: 0.083..  Accuracy: 0.554
Epoch: 1/5..  Validation Loss: 0.043..  Accuracy: 0.670
Epoch: 1/5..  Validation Loss: 0.028..  Accuracy: 0.720
Epoch: 2/5..  Validation Loss: 0.048..  Accuracy: 0.722
Epoch: 2/5..  Validation Loss: 0.024..  Accuracy: 0.765
Epoch: 2/5..  Validation Loss: 0.022..  Accuracy: 0.803
Epoch: 2/5..  Validation Loss: 0.036..  Accuracy: 0.813
Epoch: 2/5..  Validation Loss: 0.030..  Accuracy: 0.813
Epoch: 3/5..  Validation Loss: 0.027..  Accuracy: 0.813
Epoch: 3/5..  Validation Loss: 0.029..  Accuracy: 0.821
Epoch: 3/5..  Validation Loss: 0.022..  Accuracy: 0.813
Epoch: 3/5..  Validation Loss: 0.015..  Accuracy: 0.820
Epoch: 3/5..  Validation Loss: 0.020..  Accuracy: 0.832
Epoch: 4/5..  Validation Loss: 0.006..  Accuracy: 0.836
Epoch: 4/5..  Validation Loss: 0.007..  Accuracy: 0.825
Epoch: 4/5..  Validation Loss: 0.018..  Accuracy

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set
def validation(device='cpu'):
    Cuda=torch.cuda.is_available()
    if device and Cuda:
        model.cuda()
    accuracy = 0.0
    test_loss = 0.0
    for images, labels in iter(testloader):
        if device and Cuda:
            images = images.cuda()
            labels = labels.cuda()
        else:
            images = Variable(images)
            labels = Variable(labels)
        with torch.no_grad():
            result = model.forward(images)
            test_loss += criterion(result, labels).data.item()
            ps = torch.exp(result).data
            equality = (labels.data == ps.max(1)[1])
            accuracy += equality.type_as(torch.FloatTensor()).mean()
    print("Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
          "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
    
validation('gpu')


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
batch_size=64
model.class_to_idx = dataloaders.dataset.class_to_idx
model.epochs = 5
checkpoint = {'input_size': [3, 224, 224],
              'batch_size': dataloaders.batch_size,
              'output_size': 102,
              'state_dict': model.state_dict(),
              'optimizer_dict':optimizer.state_dict(),
              'class_to_idx': model.class_to_idx,
              'epoch': model.epochs}
torch.save(checkpoint, 'my_checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [12]:
# TODO: load the checkpoint
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = models.vgg16()
    initial_input= model.classifier[0].in_features
    classifier = nn.Sequential(OrderedDict([
                              ('fc1', nn.Linear(initial_input, 512)),
                              ('relu', nn.ReLU()),
                              ('drpot', nn.Dropout(p=0.5)),
                              ('hidden', nn.Linear(512, 100)),                       
                              ('fc2', nn.Linear(100, 102)),
                              ('output', nn.LogSoftmax(dim=1)),
                              ]))
    
    model.classifier = classifier
    model.load_state_dict(checkpoint['state_dict'])
    loaded_model, class_to_idx = load_checkpoint('my_checkpoint.pth')
    idx_to_class = { v : k for k,v in class_to_idx.items()}
    return model, checkpoint['class_to_idx']

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [13]:
from PIL import Image
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    Image_Size=256,256
    image.thumbnail(Image_Size, Image.ANTIALIAS)
    image=image.crop((128 - 112,128-112,128+112,128 + 112))
    My_Image = np.array(image)
    My_Image = My_Image/255
        
    X1=My_Image[:,:,0]
    X2=My_Image[:,:,1]
    X3=My_Image[:,:,2]
    
    My_Image[:,:,0]=(X1-0.485)/(0.229) 
    My_Image[:,:,1]=(X2-0.456)/(0.224)
    My_Image[:,:,2]=(X3-0.406)/(0.225)
    
    My_Image = np.transpose(My_Image, (2,0,1))
    return My_Image

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [14]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [15]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    # TODO: Implement the code to predict the class from an image file
    My_Image = torch.FloatTensor([process_image(Image.open(image_path))])
    if torch.cuda.is_available():
        model = model.cuda()
    model.eval()
    if torch.cuda.is_available():
        Result = model.forward(Variable(My_Image.cuda()))
    else:
        Result = model.forward(Variable(My_Image))
    PS = torch.exp(Result).data.numpy()[0]
    Top_Index = np.argsort(ps)[-topk:][::-1] 
    Top_Class = [idx_to_class[x] for x in Top_Index]
    Probability = PS[Top_Index]

    return Probability, Top_Class

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [18]:
# TODO: Display an image along with the top 5 classes
def print_predict(classes, probs):
    predictions = list(zip(classes, probs))
    for i in range(len(predictions)):
        print('{} : {:.3%}'.format(predictions[i][0], predictions[i][1]))
    PROB,CLASS = predict(image_path, model, topk=5)
    view_classify(img, PROB, CLS, cat_to_name)